In [1]:
import pandas as pd
import ast
import textwrap

df = pd.read_csv("trump_speaker_rows.csv")

row_idx = 12

utt = df.iloc[row_idx]["utt"]
utt = ast.literal_eval(utt) if isinstance(utt, str) else utt

speakers = df.iloc[row_idx]["speaker"]
speakers = ast.literal_eval(speakers) if isinstance(speakers, str) else speakers

print(f"\n🗣️  Original Dialogue #{row_idx}\n" + "=" * 40 + "\n")

previous_speaker = None
dialogue = ""
for spk, line in zip(speakers, utt):
    dialogue += f"{spk.upper()}:\n{line}\n\n"
    if spk != previous_speaker:
        print(f"\n{spk.upper()}:")
        previous_speaker = spk
    wrapped = textwrap.fill(line, width=90, initial_indent="  ", subsequent_indent="  ")
    print(wrapped)

print("\n" + "=" * 40)


🗣️  Original Dialogue #12


DAVID GREENE, HOST:
  A different set of judges will crown a new Miss USA next month, but you will not see
  this coronation on Univision. The Spanish-language network has canceled the telecast
  after Donald Trump made controversial comments about Mexican immigrants. He is part-
  owner of the Miss Universe Organization, which produces Miss USA. Trump made his
  comments while announcing his presidential campaign. Alexandra Starr from NPR's Code
  Switch team reports.

ALEXANDRA STARR, BYLINE:
  If you follow politics or watch television, you have probably heard Donald Trump's
  comments about immigrants who come from Mexico and other Latin American countries.

DONALD TRUMP:
  They're sending people that have lots of problems, and they're bringing those problems
  with us. They're bringing drugs. They're bringing crime. They're rapists. And some, I
  assume, are good people.

ALEXANDRA STARR, BYLINE:
  Despite that small note of conciliation, Trump's comme

In [2]:
from ollama import Client

client = Client()

obama_df = pd.read_csv("obama_sent.csv")
obama_sentences = [s.strip() for s in obama_df["sentence"] if isinstance(s, str)]

obama_examples_str = "\n".join(f"- {s}" for s in obama_sentences)

prompt = f"""\
You are a political speech transformation assistant.

Your task is to **rewrite only the utterances by DONALD TRUMP** into the speaking style of **Barack Obama**, while **preserving the original meaning** of each utterance.

Barack Obama's style is measured, thoughtful, articulate, and often includes rhetorical balance and emphasis on unity or principle. Donald Trump's style is often blunt and confrontational — you must rewrite his parts so that they sound as if Obama had delivered them, **without changing their core message or factual content**.

The conversation involves multiple speakers. Do **not change anything spoken by other participants** (e.g. Rachel Martin, Hillary Clinton, Scott Detrow). Maintain the order and speaker labels exactly as in the original.

**Return the transformed dialogue using the same format**:

SPEAKER NAME (in uppercase):  
Text of the utterance.

Only the style of Trump’s lines should change. All other content and structure must remain intact.

To guide your rewrite, here are some examples of Barack Obama’s speaking style:

{obama_examples_str}
"""

messages = [
    {"role": "user", "content": prompt},
    {"role": "user", "content": dialogue}
]

response = client.chat(model="gemma2:9b", messages=messages)
print(response["message"]["content"])

DAVID GREENE, HOST:
A different set of judges will crown a new Miss USA next month, but you will not see this coronation on Univision. The Spanish-language network has canceled the telecast after Donald Trump made controversial comments about Mexican immigrants. He is part-owner of the Miss Universe Organization, which produces Miss USA. Trump made his comments while announcing his presidential campaign. Alexandra Starr from NPR's Code Switch team reports.

ALEXANDRA STARR, BYLINE:
If you follow politics or watch television, you have probably heard Donald Trump's comments about immigrants who come from Mexico and other Latin American countries.

DONALD TRUMP:
There are issues that we need to address regarding immigration, and I believe it is important for us to be clear-eyed about the challenges posed by some of those coming across our borders.  We need to ensure that our nation's security is protected, and that means taking a careful look at immigration policies.

ALEXANDRA STARR, BYL

In [3]:
output_text = response["message"]["content"]

filename = f"obama_style_rewrite_2.txt"

with open(filename, "w", encoding="utf-8") as f:
    f.write(output_text)

print(f"Response saved to {filename}")

Response saved to obama_style_rewrite_2.txt
